# Exemplos `rvest`

In [1]:
renv::init()
renv::install()

- There are no packages to install.


## Exemplos de extracão de Top 250 filmes do IMDB (https://www.imdb.com/chart/top)

In [2]:
imdb_url <- "https://www.imdb.com/"
top_250_url <- "chart/top"

top_250_html <- rvest::read_html(paste0(imdb_url, top_250_url))

In [3]:
movie_urls <- top_250_html |>
  rvest::html_nodes(".cli-children a") |>
  rvest::html_attr("href")

movie_urls[1:5]

[1] "/title/tt0111161/?ref_=chttp_t_1" "/title/tt0068646/?ref_=chttp_t_2"
[3] "/title/tt0468569/?ref_=chttp_t_3" "/title/tt0071562/?ref_=chttp_t_4"
[5] "/title/tt0050083/?ref_=chttp_t_5"

In [4]:
file_conn <- file("../../data/rvest/top_250_filmes.txt", open = "a")

In [5]:
header <- c("title", "rating", "score", "director")
writeLines(paste(header, collapse = ";"), file_conn, sep = "\n")

for (movie_url in movie_urls) {
  movie_html <- rvest::read_html(paste0(imdb_url, movie_url))

  title <- movie_html |>
    rvest::html_nodes("h1") |>
    rvest::html_text()

  rating <- movie_html |>
    rvest::html_nodes("h1") |>
    rvest::html_text()

  score <- movie_html |>
    rvest::html_nodes("h1") |>
    rvest::html_text()

  director <- movie_html |>
    rvest::html_nodes("h1") |>
    rvest::html_text()

  line <- c(title, rating, score, director)
  writeLines(paste(line, collapse = ";"), file_conn, sep = "\n")
  Sys.sleep(1)
}

In [6]:
close(file_conn)

## Exemplos de extracão de citacões (https://quotes.toscrape.com/)

Utilizar o `rvest` para relizar a requisicão e capturar o `html` da página

In [7]:
html <- rvest::read_html("https://quotes.toscrape.com/")

In [8]:
quotes <- html |> 
  rvest::html_nodes(".text") |>
  rvest::html_text()

quotes[1:2]

[1] "“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”"
[2] "“It is our choices, Harry, that show what we truly are, far more than our abilities.”"

Abrindo uma conexão de arquivo para salvar os resultados

In [9]:
file_conn <- file("../../data/rvest/citacoes_10_paginas.txt", open = "a")

In [10]:
# Padrão de url das citacões
url_pattern <- "https://quotes.toscrape.com/page/%d"
urls <- sprintf(url_pattern, 1:10)

# Criando `header` do arquivo csv
header <- c("author", "quote")
writeLines(paste(header, collapse = ";"), file_conn, sep = "\n")

for(url in urls) {
  # Requisicão do html
  html <- rvest::read_html(url)

  authors <- html |>
    rvest::html_nodes(".author") |>
    rvest::html_text()

  quotes <- html |>
    rvest::html_nodes(".text") |>
    rvest::html_text()

  writeLines(paste0(authors, ";", quotes, collapse = "\n"), file_conn, sep = "")
  Sys.sleep(1)
}

Fechar arquivo

In [11]:
close(file_conn)

## Exemplos de extracão de citacões com preenchimento de formulário (https://quotes.toscrape.com/search.aspx)

##### Exemplo único

In [12]:
html <- rvest::read_html("https://quotes.toscrape.com/search.aspx")

hidden_inputs <- html |>
  rvest::html_nodes("input[type='hidden']") |>
  (\(x) setNames(
    rvest::html_attr(x, "value"),
    rvest::html_attr(x, "name")
  ))()

response <- httr::POST(
  url = "https://quotes.toscrape.com/filter.aspx",
  httr::add_headers(
    `Content-Type` = "application/x-www-form-urlencoded"
  ),
  body = list(
    author = "Haruki Murakami",
    tag = "thought",
    `__VIEWSTATE` = hidden_inputs["__VIEWSTATE"]
  ),
  encode = "form"
)

In [13]:
quotes <- response |>
  rvest::read_html() |>
  rvest::html_nodes(".quote .content") |>
  rvest::html_text()

quotes

[1] "“If you only read the books that everyone else is reading, you can only think what everyone else is thinking.”"

##### Exemplo vários

In [14]:
html <- rvest::read_html("https://quotes.toscrape.com/search.aspx")

authors <- html |>
  rvest::html_nodes("#author option") |>
  rvest::html_attr("value")

authors <- Filter(\(x) !is.na(x), authors)

hidden_inputs <- html |>
  rvest::html_nodes("input[type='hidden']") |>
  (\(x) setNames(
    rvest::html_attr(x, "value"),
    rvest::html_attr(x, "name")
  ))()

In [15]:
file_conn <- file("../../data/rvest/citacoes_filtro.txt", open = "a")

In [16]:
header <- c("author", "quotes", "tag")
writeLines(paste(header, collapse = ";"), file_conn, sep = "\n")

for(author in authors) {
  response <- httr::POST(
    url = "https://quotes.toscrape.com/filter.aspx",
    httr::add_headers(
      `Content-Type` = "application/x-www-form-urlencoded"
    ),
    body = list(
      author = author,
      `__VIEWSTATE` = hidden_inputs["__VIEWSTATE"]
    ),
    encode = "form"
  )

  tags <- response |>
    rvest::read_html() |>
    rvest::html_nodes("#tag option")  |>
    rvest::html_attr("value")

  for(tag in tags) {
    response <- httr::POST(
      url = "https://quotes.toscrape.com/filter.aspx",
      httr::add_headers(
        `Content-Type` = "application/x-www-form-urlencoded"
      ),
      body = list(
        author = author,
        tag = tag,
        `__VIEWSTATE` = hidden_inputs["__VIEWSTATE"]
      ),
      encode = "form"
    )

    quotes <- response |>
      rvest::read_html() |>
      rvest::html_nodes(".quote .content") |>
      rvest::html_text()

    quotes <- Filter(\(x) !identical(x, character(0)), quotes)

    for (quote in quotes) {
      line = c(author, quote, tag)
      writeLines(paste0(line, collapse = ";"), file_conn, sep = "\n")
    }

  }
}

In [17]:
close(file_conn)

## Exemplos de extracão de citacões com login (https://quotes.toscrape.com/login)

In [18]:
rvest::read_html("https://quotes.toscrape.com") |>
  rvest::html_text() |>
  cat()

Quotes to Scrape
    
        
            
                
                    Quotes to Scrape
                
            
            
                
                
                    Login
                
                
            
        
    


    

    
        “The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
        by Albert Einstein
        (about)
        
        
            Tags:
            change
            
            deep-thoughts
            
            thinking
            
            world
            
        
    

    
        “It is our choices, Harry, that show what we truly are, far more than our abilities.”
        by J.K. Rowling
        (about)
        
        
            Tags:
            abilities
            
            choices
            
        
    

    
        “There are only two ways to live your life. One is as though nothing is a miracle. The other is as t

In [19]:
session <- rvest::session("https://quotes.toscrape.com/login")

hidden_inputs <- session |>
  rvest::html_nodes("input[type='hidden']") |>
  (\(x) setNames(
    rvest::html_attr(x, "value"),
    rvest::html_attr(x, "name")
  ))()

hidden_inputs


csrf_token 
"tDHIVJydewFGWbcLnlmUfzaupZMrBvjhNTQPsYOgRSEKxqCkXoiA"

In [20]:
form <- session |>
  rvest::html_form() |>
  (\(x) x[[1]])() |>
  rvest::html_form_set(
    username = "Username",
    password = "Password"
  )

session <- rvest::session_submit(session, form)
session |>
  rvest::session_jump_to("https://quotes.toscrape.com") |>
  rvest::read_html() |>
  rvest::html_text() |>
  cat()

Quotes to Scrape
    
        
            
                
                    Quotes to Scrape
                
            
            
                
                
                    Logout
                
                
            
        
    


    

    
        “The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
        by Albert Einstein
        (about) - (Goodreads page)
        
        
            Tags:
            change
            
            deep-thoughts
            
            thinking
            
            world
            
        
    

    
        “It is our choices, Harry, that show what we truly are, far more than our abilities.”
        by J.K. Rowling
        (about) - (Goodreads page)
        
        
            Tags:
            abilities
            
            choices
            
        
    

    
        “There are only two ways to live your life. One is as though 